# Object2Vec を用いたドキュメントの分散表現取得

#### ノートブックに含まれる内容

- Object2Vec の使い方

#### ノートブックで使われている手法の詳細

- アルゴリズム: Object2Vec
- データ: Wikipedia データ


## 概要

このノートブックでは，Object2Vec でドキュメントの分散表現を獲得する学習と推論を行います．Object2Vec は，2 種類のインプットをとります．その 2 種類のインプットをそれぞれ分散表現に落とした上で，それらを比較して最終的なスコア / ラベルを予測します．インプットとして，文章や単語，ID 等を自由に取ることが可能です．アルゴリズム概要については，[Amazon SageMaker Object2Vec の概要](https://aws.amazon.com/jp/blogs/news/introduction-to-amazon-sagemaker-object2vec/)をご覧ください．

ここでは，Wikipedia のドキュメントをもとに，ドキュメント内のあるセンテンスと，ドキュメントから当該センテンスを抜いたもののペアを入力とします．このペアに対しては，正例のラベル 1 を付与します．学習用に準備するデータは，正例データのみです．Object2Vec の学習時の`negative_sampling_rate` を 1 以上の整数値にすることで，1 つの正例サンプルに対して指定した数ぶんの負例サンプルが学習時に自動生成されます．負例データは，センテンスとドキュメントをランダムに組み合わせたものとなります．

これを図で表すと，以下の通りになります．

<img src="doc_embedding_illustration.png" width="800">

Object2Vec を分散表現を獲得するために用いる場合は，パフォーマンス向上のために，他に疎行列による勾配の更新，Embedding レイヤーの重みのエンコーダ間共有，また比較演算子のカスタマイズといったいくつかのハイパーパラメタを利用することが可能です．詳細は[こちらのブログ](https://aws.amazon.com/jp/blogs/news/amazon-sagemaker-object2vec-adds-new-features-that-support-automatic-negative-sampling-and-speed-up-training/)をご覧ください．

## データの準備

Facebook AI が提供している Wikipedia のコーパスデータを用いて，Object2Vec 用に加工します．以下，前処理のスニペットが多く並びますが，単に Object2Vec の入力形式に整形しているだけです．入力形式については，[こちら](https://docs.aws.amazon.com/ja_jp/sagemaker/latest/dg/object2vec-training-formats.html)にまとまっている通りです．`in0` は `encoder0` の入力，`in1` は `encoder1` の入力になります．ここでは，センテンスとドキュメントを語彙ファイルで数値 ID にエンコードしなおした配列が，それぞれのエンコーダーに与えられます．

```
{"label": 1, "in0": [774, 14, 21, 206], "in1": [21, 366, 125]}
{"label": 1, "in0": [236, 4, 227, 391, 521, 881], "in1": [31, 1369]}
...
```

In [ ]:
%%bash

DATANAME="wikipedia"
DATADIR="/tmp/wiki"

mkdir -p "${DATADIR}"

if [ ! -f "${DATADIR}/${DATANAME}_train250k.txt" ]
then
    echo "Downloading wikipedia data"
    wget --quiet -c "https://dl.fbaipublicfiles.com/starspace/wikipedia_train250k.tgz" -O "${DATADIR}/${DATANAME}_train.tar.gz"
    tar -xzvf "${DATADIR}/${DATANAME}_train.tar.gz" -C "${DATADIR}"
    wget --quiet -c "https://dl.fbaipublicfiles.com/starspace/wikipedia_devtst.tgz" -O "${DATADIR}/${DATANAME}_test.tar.gz"
    tar -xzvf "${DATADIR}/${DATANAME}_test.tar.gz" -C "${DATADIR}"
fi


In [ ]:
datadir = '/tmp/wiki'

In [ ]:
!ls /tmp/wiki

In [ ]:
!pip install jsonlines

In [ ]:
# note: please run on python 3 kernel

import os
import random

import math
import scipy
import numpy as np

import re
import string
import json, jsonlines

from collections import defaultdict
from collections import Counter

from itertools import chain, islice

from nltk.tokenize import TreebankWordTokenizer
from sklearn.preprocessing import normalize

## sagemaker api
import sagemaker, boto3
from sagemaker.session import s3_input
from sagemaker.predictor import json_serializer, json_deserializer

sess = sagemaker.session.Session()

In [ ]:
BOS_SYMBOL = "<s>"
EOS_SYMBOL = "</s>"
UNK_SYMBOL = "<unk>"
PAD_SYMBOL = "<pad>"
PAD_ID = 0
TOKEN_SEPARATOR = " "
VOCAB_SYMBOLS = [PAD_SYMBOL, UNK_SYMBOL, BOS_SYMBOL, EOS_SYMBOL]


##### utility functions for preprocessing
def get_article_iter_from_file(fname):
    with open(fname) as f:
        for article in f:
            yield article

def get_article_iter_from_channel(channel, datadir='/tmp/wiki'):
    if channel == 'train':
        fname = os.path.join(datadir, 'wikipedia_train250k.txt')
        return get_article_iter_from_file(fname)
    else:
        iterlist = []
        suffix_list = ['train250k.txt', 'test10k.txt', 'dev10k.txt', 'test_basedocs.txt']
        for suffix in suffix_list:
            fname = os.path.join(datadir, 'wikipedia_'+suffix)
            iterlist.append(get_article_iter_from_file(fname))
        return chain.from_iterable(iterlist)


def readlines_from_article(article):
    return article.strip().split('\t')


def sentence_to_integers(sentence, word_dict, trim_size=None):
    """
    Converts a string of tokens to a list of integers
    """
    if not trim_size:
        return [word_dict[token] if token in word_dict else 0 for token in get_tokens_from_sentence(sentence)]
    else:
        integer_list = []
        for token in get_tokens_from_sentence(sentence):
            if len(integer_list) < trim_size:
                if token in word_dict:
                    integer_list.append(word_dict[token])
                else:
                    integer_list.append(0)
            else:
                break
        return integer_list


def get_tokens_from_sentence(sent):
    """
    Yields tokens from input string.

    :param line: Input string.
    :return: Iterator over tokens.
    """
    for token in sent.split():
        if len(token) > 0:
            yield normalize_token(token)


def get_tokens_from_article(article):
    iterlist = []
    for sent in readlines_from_article(article):
        iterlist.append(get_tokens_from_sentence(sent))
    return chain.from_iterable(iterlist)


def normalize_token(token):
    token = token.lower()
    if all(s.isdigit() or s in string.punctuation for s in token):
        tok = list(token)
        for i in range(len(tok)):
            if tok[i].isdigit():
                tok[i] = '0'
        token = "".join(tok)
    return token

In [ ]:
# function to build vocabulary

def build_vocab(channel, num_words=50000, min_count=1, use_reserved_symbols=True, sort=True):
    """
    Creates a vocabulary mapping from words to ids. Increasing integer ids are assigned by word frequency,
    using lexical sorting as a tie breaker. The only exception to this are special symbols such as the padding symbol
    (PAD).

    :param num_words: Maximum number of words in the vocabulary.
    :param min_count: Minimum occurrences of words to be included in the vocabulary.
    :return: word-to-id mapping.
    """
    vocab_symbols_set = set(VOCAB_SYMBOLS)
    raw_vocab = Counter()
    for article in get_article_iter_from_channel(channel):
        article_wise_vocab_list = list()
        for token in get_tokens_from_article(article):
            if token not in vocab_symbols_set:
                article_wise_vocab_list.append(token)
        raw_vocab.update(article_wise_vocab_list)

    print("Initial vocabulary: {} types".format(len(raw_vocab)))

    # For words with the same count, they will be ordered reverse alphabetically.
    # Not an issue since we only care for consistency
    pruned_vocab = sorted(((c, w) for w, c in raw_vocab.items() if c >= min_count), reverse=True)
    print("Pruned vocabulary: {} types (min frequency {})".format(len(pruned_vocab), min_count))

    # truncate the vocabulary to fit size num_words (only includes the most frequent ones)
    vocab = islice((w for c, w in pruned_vocab), num_words)

    if sort:
        # sort the vocabulary alphabetically
        vocab = sorted(vocab)
    if use_reserved_symbols:
        vocab = chain(VOCAB_SYMBOLS, vocab)

    word_to_id = {word: idx for idx, word in enumerate(vocab)}

    print("Final vocabulary: {} types".format(len(word_to_id)))

    if use_reserved_symbols:
        # Important: pad symbol becomes index 0
        assert word_to_id[PAD_SYMBOL] == PAD_ID
    
    return word_to_id

In [ ]:
# build vocab dictionary

def build_vocabulary_file(vocab_fname, channel, num_words=50000, min_count=1, 
                          use_reserved_symbols=True, sort=True, force=False):
    if not os.path.exists(vocab_fname) or force:
        w_dict = build_vocab(channel, num_words=num_words, min_count=min_count, 
                             use_reserved_symbols=True, sort=True)
        with open(vocab_fname, "w") as write_file:
            json.dump(w_dict, write_file)

channel = 'train'
min_count = 5
vocab_fname = os.path.join(datadir, 'wiki-vocab-{}250k-mincount-{}.json'.format(channel, min_count))

build_vocabulary_file(vocab_fname, channel, num_words=500000, min_count=min_count, force=True)

In [ ]:
print("Loading vocab file {} ...".format(vocab_fname))

with open(vocab_fname) as f:
    w_dict = json.load(f)
    print("The vocabulary size is {}".format(len(w_dict.keys())))

In [ ]:
# Functions to build training data 
# Tokenize wiki articles to (sentence, document) pairs
def generate_sent_article_pairs_from_single_article(article, word_dict):
    sent_list = readlines_from_article(article)
    art_len = len(sent_list)
    idx = random.randint(0, art_len-1)
    wrapper_text_idx = list(range(idx)) + list(range((idx+1) % art_len, art_len))
    wrapper_text_list = sent_list[:idx] + sent_list[(idx+1) % art_len : art_len]
    wrapper_tokens = []
    for sent1 in wrapper_text_list:
        wrapper_tokens += sentence_to_integers(sent1, word_dict)
    sent_tokens = sentence_to_integers(sent_list[idx], word_dict)
    yield {'in0':sent_tokens, 'in1':wrapper_tokens, 'label':1}


def generate_sent_article_pairs_from_single_file(fname, word_dict):
    with open(fname) as reader:
        iter_list = []
        for article in reader:
            iter_list.append(generate_sent_article_pairs_from_single_article(article, word_dict))
    return chain.from_iterable(iter_list)

In [ ]:
# Build training data

# Generate integer positive labeled data
train_prefix = 'train250k'
fname = "wikipedia_{}.txt".format(train_prefix)
outfname = os.path.join(datadir, '{}_tokenized.jsonl'.format(train_prefix))
counter = 0

with jsonlines.open(outfname, 'w') as writer:
    for sample in generate_sent_article_pairs_from_single_file(os.path.join(datadir, fname), w_dict):
        writer.write(sample)
        counter += 1
        
print("Finished generating {} data of size {}".format(train_prefix, counter))

In [ ]:
# Shuffle training data
!shuf {outfname} > {train_prefix}_tokenized_shuf.jsonl

In [ ]:
## Function to generate dev/test data (with both positive and negative labels)

def generate_pos_neg_samples_from_single_article(word_dict, article_idx, article_buffer, negative_sampling_rate=1):
    sample_list = []
    # generate positive samples
    sent_list = readlines_from_article(article_buffer[article_idx])
    art_len = len(sent_list)
    idx = random.randint(0, art_len-1)
    wrapper_text_idx = list(range(idx)) + list(range((idx+1) % art_len, art_len))
    wrapper_text_list = sent_list[:idx] + sent_list[(idx+1) % art_len : art_len]
    wrapper_tokens = []
    for sent1 in wrapper_text_list:
        wrapper_tokens += sentence_to_integers(sent1, word_dict)
    sent_tokens = sentence_to_integers(sent_list[idx], word_dict)
    sample_list.append({'in0':sent_tokens, 'in1':wrapper_tokens, 'label':1})
    # generate negative sample
    buff_len = len(article_buffer)
    sampled_inds = np.random.choice(list(range(article_idx)) + list(range((article_idx+1) % buff_len, buff_len)), 
                                    size=negative_sampling_rate)
    for n_idx in sampled_inds:
        other_article = article_buffer[n_idx]
        context_list = readlines_from_article(other_article)
        context_tokens = []
        for sent2 in context_list:
            context_tokens += sentence_to_integers(sent2, word_dict)
        sample_list.append({'in0': sent_tokens, 'in1':context_tokens, 'label':0})
    return sample_list

In [ ]:
# Build dev and test data
for data in ['dev10k', 'test10k']:
    fname = os.path.join(datadir,'wikipedia_{}.txt'.format(data))
    test_nsr = 5
    outfname = '{}_tokenized-nsr{}.jsonl'.format(data, test_nsr)
    article_buffer = list(get_article_iter_from_file(fname))
    sample_buffer = []
    for article_idx in range(len(article_buffer)):
        sample_buffer += generate_pos_neg_samples_from_single_article(w_dict, article_idx, 
                                                                      article_buffer, 
                                                                      negative_sampling_rate=test_nsr)
    with jsonlines.open(outfname, 'w') as writer:
        writer.write_all(sample_buffer)

これで学習用，テスト用のデータセットが作成できたので，中身を確認してみましょう．`in0` にセンテンスが，`in1` にドキュメントが，それぞれ数値エンコードされた配列として格納されており，そのあとに正例である 1 のラベルが付与された jsonl の形式です．冒頭で述べたように，アルゴリズム内で自動生成されるため，学習データに負例データは含まれていません．

In [ ]:
!head -n 3 train250k_tokenized_shuf.jsonl

## データのロード

In [ ]:
TRAIN_DATA="train250k_tokenized_shuf.jsonl"
DEV_DATA="dev10k_tokenized-nsr{}.jsonl".format(test_nsr)
TEST_DATA="test10k_tokenized-nsr{}.jsonl".format(test_nsr)

# NOTE: define your s3 bucket and key here
S3_BUCKET = sess.default_bucket()
S3_KEY = 'object2vec-doc2vec'



In [ ]:
%%bash -s "$TRAIN_DATA" "$DEV_DATA" "$TEST_DATA" "$S3_BUCKET" "$S3_KEY"

aws s3 cp "$1" s3://$4/$5/input/train/
aws s3 cp "$2" s3://$4/$5/input/validation/
aws s3 cp "$3" s3://$4/$5/input/test/

In [ ]:
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

region = boto3.Session().region_name
print("Your notebook is running on region '{}'".format(region))
 
role = get_execution_role()
print("Your IAM role: '{}'".format(role))

container = get_image_uri(region, 'object2vec')
print("The image uri used is '{}'".format(container))

print("Using s3 buceket: {} and key prefix: {}".format(S3_BUCKET, S3_KEY))

In [ ]:
## define input channels

s3_input_path = os.path.join('s3://', S3_BUCKET, S3_KEY, 'input')

s3_train = s3_input(os.path.join(s3_input_path, 'train', TRAIN_DATA), 
                    distribution='ShardedByS3Key', content_type='application/jsonlines')

s3_valid = s3_input(os.path.join(s3_input_path, 'validation', DEV_DATA), 
                    distribution='ShardedByS3Key', content_type='application/jsonlines')

s3_test = s3_input(os.path.join(s3_input_path, 'test', TEST_DATA), 
                   distribution='ShardedByS3Key', content_type='application/jsonlines')

In [ ]:
## define output path
output_path = os.path.join('s3://', S3_BUCKET, S3_KEY, 'models')

## モデルの学習を実行

ハイパーパラメタを以下のように設定して，モデルの学習を実行します．学習には 15 分ほどかかります．ハイパーパラメタの詳細については，[こちら](https://docs.aws.amazon.com/ja_jp/sagemaker/latest/dg/object2vec-hyperparameters.html)をご参照ください．

In [ ]:
# Define training hyperparameters

hyperparameters = {
      "_kvstore": "device",
      "_num_gpus": 'auto',
      "_num_kv_servers": "auto",
      "bucket_width": 0,
      "dropout": 0.4,
      "early_stopping_patience": 2,
      "early_stopping_tolerance": 0.01,
      "enc0_layers": "auto",
      "enc0_max_seq_len": 50,
      "enc0_network": "pooled_embedding",
      "enc0_pretrained_embedding_file": "",
      "enc0_token_embedding_dim": 300,
      "enc0_vocab_size": 267522,
      "enc1_network": "enc0",
      "enc_dim": 300,
      "epochs": 20,
      "learning_rate": 0.01,
      "mini_batch_size": 512,
      "mlp_activation": "relu",
      "mlp_dim": 512,
      "mlp_layers": 2,
      "num_classes": 2,
      "optimizer": "adam",
      "output_layer": "softmax",
      "weight_decay": 0
}


hyperparameters['negative_sampling_rate'] = 3
hyperparameters['tied_token_embedding_weight'] = "true"
hyperparameters['comparator_list'] = "hadamard"
hyperparameters['token_embedding_storage_type'] = 'row_sparse'

    
# get estimator
doc2vec = sagemaker.estimator.Estimator(container,
                                          role, 
                                          train_instance_count=1, 
                                          train_instance_type='ml.p2.xlarge',
                                          output_path=output_path,
                                          sagemaker_session=sess)

In [ ]:
# set hyperparameters
doc2vec.set_hyperparameters(**hyperparameters)

# fit estimator with data
doc2vec.fit({'train': s3_train, 'validation':s3_valid, 'test':s3_test})

## モデルの推論を実行

推論を行うために，Estimateor オブジェクトからモデルオブジェクトを作成した上で，モデルをエンドポイントにデプロイします．deploy() メソッドでは，デプロイ先エンドポイントのインスタンス数，インスタンスタイプを指定します．モデルのデプロイには 10 分程度時間がかかります．

In [ ]:
# deploy model

doc2vec_model = doc2vec.create_model(
                        serializer=json_serializer,
                        deserializer=json_deserializer,
                        content_type='application/json')

predictor = doc2vec_model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

エンドポイントができたら，テスト用のデータを準備して，実際に分散表現を取得してみましょう．分散表現を取得する際には，以下のようなデータをリクエストボディとして準備します．`in0` であれば `encoder0` が，`in1` であれば `encoder1` が，分散表現の獲得のために使用されます．ただし今回は，`tied_token_embedding_weight` を `True` にしており，`encoder0` と `encoder1` は共通のものが使われるため，`in0` でも `in1` でも帰ってくる結果は同一のものになります．

ここでは推論に json フォーマットを使用していますが，jsonlines フォーマットを使用することも可能です．詳細については，[こちら](https://docs.aws.amazon.com/ja_jp/sagemaker/latest/dg/object2vec-encoder-embeddings.html)をご覧ください．

```
{
  "instances" : [
    {"in0": [6, 17, 606, 19, 53, 67, 52, 12, 5, 10, 15, 10178, 7, 33, 652, 80, 15, 69, 821, 4]},
    {"in0": [22, 1016, 32, 13, 25, 11, 5, 64, 573, 45, 5, 80, 15, 67, 21, 7, 9, 107, 4]},
    {"in0": [774, 14, 21, 206]}
  ]
}
```


In [ ]:
# get embeddings for sentence and ground-truth article pairs
sentences = []
gt_articles = []
pairs = []
for data in read_jsonline(test_fpath):
    if data['label'] == 1:
        sentences.append({'in0': data['in0']})
        gt_articles.append({'in0': data['in1']})
        pairs.append({'in0': data['in0'], 'in1': data['in1']})

In [ ]:
payload = {'instances': sentences[:3]}
response = send_payload(predictor, payload)
print(response)

また以下のように `in0` と `in1` を両方入力データに含めることで，1/0 ラベルの推論結果を取得することが可能になります．

```
{
  "instances" : [
    {"in0": [774, 14, 21, 206], "in1": [21, 366, 125]}
    {"in0": [236, 4, 227, 391, 521, 881], "in1": [31, 1369]}
  ]
}
```

In [ ]:
payload = {'instances': pairs[:3]}
response = send_payload(predictor, payload)
print(response)

## エンドポイントの削除

全て終わったら，エンドポイントを削除します．

In [ ]:
predictor.delete_endpoint()